In [ ]:
# В блокноте производится предобработка исходных данных асмпп по одному мвн.

# Предобработка заключается в следующем:
# 1. Выделение в исходных данных отдельных рейсов.
# 2. Выделение всех возможных вариантов поостановочной трассы.
# 3. Разделение по временным периодам. Выделены следующие категории временных периодов:
#    - раб/вых -- разделение в соответсвии со справочником рабочих дней;
#    - лето/зима -- лето: с 1 мая по 30 сентября, зима с 1 октября по 30 апреля (можно менять);
#    - час суток ("ЧЧ:ММ-ЧЧ:ММ") -- привязка рейса к часу суток осуществляется по времени прибытия на первый о.п. трассы
#    Временной период задается строкой вида <сезон>-<тип дня>-<час суток>, например, "лето-раб-08:00-08:59"
# 3. Вычисление числа рейсов по каждому варианту поостановочной трассы в указанный временной период.
# 4. Вычисление посадки-высадки среднего рейса.
# 
# В результате создается набор таблиц, содержащих следующие сведения:
# 1. Таблица inout -- посадка-высадка:
#    - mvn -- название мвн;
#    - trace_id -- идентификационный номер варианта поостановочной трассы (см. табл. traces);
#    - period_type -- тип временного периодов;
#    - stop_sequence -- порядковый номер о.п.;
#    - stop_id -- идентификационный номер о.п.;
#    - stop_name -- наименование о.п.;
#    - pass_in -- посадка;
#    - pass_out -- высадка.
# 2. Таблица traces -- варианты поостановочных трасс:
#    - trace_id -- идентификационный номер варианта поостановочной трассы;
#    - n_races -- общее число рейсов в исходных данных асмпп;
#    - date_first -- дата начала первого рейса в исходных данных асмпп;
#    - date_last -- дата начала последнего рейса в исходных данных асмпп;
#    - flag -- флаг использования данных о посаждке-высадке для построения маршрутной матрицы (1 -- используется);
#    - trip_id -- перечень через запятую trip_id имеющих указанную поостановочную трассу;
#    - trace -- поостановочная трасса составленная из последовательного перечня "<stop_sequence>-<stop_id>", разделенного ",\n".
# 3. Таблица nraces -- число рейсов по типам временных интервалов:
#    - первые три поля -- категории типов временных периодов:
#      - season -- лето/зима
#      - workday -- раб/вых
#      - time -- час суток
#    - остальные поля таблицы -- варианты поостановочной трассы мвн, в названии поля указан trace_id (см. табл. traces);
#    - значения -- число рейсов в исходных данных асмпп.
# 4. Таблица pass -- число перевезенных пассажиров в среднем рейсе по типам временных интервалов:
#    - первые три поля -- категории типов временных периодов:
#      - season -- лето/зима
#      - workday -- раб/вых
#      - time -- час суток
#    - остальные поля таблицы -- варианты поостановочной трассы мвн, в названии поля указан trace_id (см. табл. traces);
#    - значения -- число пассажиров в среднем рейсе асмпп в формате "<среднее число пассажиров> ± <ср. кв. откл.>".
#


In [1]:
import os
import sys
sys._enablelegacywindowsfsencoding()
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

%load_ext iimport
%matplotlib inline

import geopandas as gpd
from sqlalchemy import create_engine
from fiona.crs import from_epsg

iimport loaded.


In [2]:
all_data_file = pd.read_excel('inout_Вечер пик.xlsx')
all_data_file

,stop_id,mvn,stop_sequence,pass_in,pass_out
0,1,26-Прямое-1201-Вечер пик,9,0.0,0.0
1,1,26-Прямое-2134-Вечер пик,9,0.0,0.0
2,1,26-Прямое-2139-Вечер пик,9,0.0,0.0
3,1,26-Прямое-4280-Вечер пик,9,0.0,0.0
4,1,26-Прямое-5252-Вечер пик,9,0.0,0.0
5,1,28-Прямое-1220-Вечер пик,11,0.0,0.0
6,1,28-Прямое-1225-Вечер пик,11,2.0,0.0
7,1,28-Прямое-3024-Вечер пик,11,1.0,0.0
8,1,28-Прямое-3025-Вечер пик,11,0.0,2.0
9,1,28-Прямое-4555-Вечер пик,11,0.0,0.0


In [3]:
all_data_file["mvn"]=all_data_file.mvn.str.replace('К-', 'К_',1)
all_data_file["mvn"]=all_data_file.mvn.str.replace('Трл-', 'Трл_',1)
all_data_file["mvn"]=all_data_file.mvn.str.replace('Тм-', 'Тм_',1)

all_data_file['route_id'] = all_data_file["mvn"].str.split("-", n = 1, expand = True)[0]

all_data_file['direction'] = all_data_file["mvn"].str.split("-", n = 2, expand = True)[1]



all_data_file

,stop_id,mvn,stop_sequence,pass_in,pass_out,route_id,direction
0,1,26-Прямое-1201-Вечер пик,9,0.0,0.0,26,Прямое
1,1,26-Прямое-2134-Вечер пик,9,0.0,0.0,26,Прямое
2,1,26-Прямое-2139-Вечер пик,9,0.0,0.0,26,Прямое
3,1,26-Прямое-4280-Вечер пик,9,0.0,0.0,26,Прямое
4,1,26-Прямое-5252-Вечер пик,9,0.0,0.0,26,Прямое
5,1,28-Прямое-1220-Вечер пик,11,0.0,0.0,28,Прямое
6,1,28-Прямое-1225-Вечер пик,11,2.0,0.0,28,Прямое
7,1,28-Прямое-3024-Вечер пик,11,1.0,0.0,28,Прямое
8,1,28-Прямое-3025-Вечер пик,11,0.0,2.0,28,Прямое
9,1,28-Прямое-4555-Вечер пик,11,0.0,0.0,28,Прямое


In [4]:
stops = pd.read_excel('stops.xlsx')
stops

,stop_id,stop_name,lat,lon
0,205,пос. Обидимо,54.306066,37.424022
1,216,Парковая,54.297935,37.441982
2,219,Больница,54.293729,37.445221
3,223,Микрорайон,54.291530,37.450947
4,226,Совхозная,54.289768,37.455553
5,235,Рынок,54.286594,37.464210
6,245,Каньшино,54.276950,37.471729
7,249,Гараж,54.273641,37.476968
8,254,Шоссейная,54.267693,37.481476
9,258,пос. Барсуки,54.262773,37.484562


In [5]:
matrix_avg = all_data_file.groupby(['route_id','stop_sequence','stop_id','direction']).mean()[['pass_in','pass_out']].reset_index()

matrix_avg.sort_values(['route_id','direction','stop_sequence'], inplace=True)

matrix_avg = matrix_avg.reset_index(drop=True)

matrix_avg['mvn'] = matrix_avg[['route_id', 'direction']].apply(lambda x: '+'.join(x), axis=1)

# matrix_avg=matrix_avg.rename(columns = {'route_id':'mvn'})

matrix_avg['trace_id'] = '1'

matrix_avg['period_type'] = 'period_type'

matrix_avg = matrix_avg.merge(stops, on='stop_id', how='left')

matrix_avg = pd.DataFrame(matrix_avg, columns=['mvn','trace_id','period_type','stop_sequence','stop_id','stop_name',
                                               'pass_in','pass_out'])

matrix_avg

,mvn,trace_id,period_type,stop_sequence,stop_id,stop_name,pass_in,pass_out
0,1+Обратное,1,period_type,1,785,Городское кладбище №3,0.500000,0.000000
1,1+Обратное,1,period_type,2,757,Осиновая гора,0.000000,0.000000
2,1+Обратное,1,period_type,3,738,Городское кладбище №2,0.750000,0.000000
3,1+Обратное,1,period_type,4,708,Городское кладбище №1,0.250000,0.000000
4,1+Обратное,1,period_type,5,710,Автоколонна №1136,1.000000,0.250000
5,1+Обратное,1,period_type,6,700,Автоколонна №1809,1.250000,0.000000
6,1+Обратное,1,period_type,7,714,Спецавтохозяйство,0.500000,0.000000
7,1+Обратное,1,period_type,8,720,Оптовый рынок,0.250000,0.000000
8,1+Обратное,1,period_type,9,711,Гипермаркет Линия,0.750000,1.000000
9,1+Обратное,1,period_type,10,717,Восточный обвод,0.000000,0.000000


In [ ]:
# from datetime import datetime
# matrix_avg.to_csv('matrix_avg-{:%Y-%m-%d-(%H-%M)}.csv'.format(datetime.now()), sep=';', encoding='cp1251', index=False)

In [ ]:
list_mvns = matrix_avg[['mvn']].drop_duplicates('mvn').reset_index(drop=True)
list_mvns

In [ ]:
for i in tqdm(range(len(list_mvns))):
    list_i = list_mvns.mvn[i]
    df = matrix_avg.copy(deep=True)
    df = df.query('@df.mvn == @list_i')
    df.to_excel('C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/mvns_files/' + list_i + '.xlsx', encoding='cp1251', index=False)


# Part C1

# !!!Для выгрузки файлов по каждому из пиков (утро, межпик, вечер) прописать соответствующее время в директории и условие в цикле!!!

In [6]:
# !!!ПРОПИСАТЬ ЗДЕСЬ!!!
timing = 'Вечер пик'

In [ ]:
# experiment = pd.read_excel('C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/обработанные/преобразованные ' 
#     + timing +'/'+'1-Обратное-471-'+ timing +'.xlsx')
# experiment

In [14]:
args = {
    # файл с предобработанными данными асмпп по одному мвн
    'source' : 'C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/mvns_files/1+Обратное.xlsx',
    # директория с результатами
    
    
    # !!!!!!
    'results' : 'C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/обработанные/преобразованные ' 
    + timing +'/',
    
    
    # тип периода
    'period-type' : None,#'лето-раб-06:00-06:59',
    # лог-файл
    'log' : None,
    # коэффициенты весовой фукции min((R/beta)**alpha, 1)
    'alpha' : 2,    
    'beta'  : 1000, # расстояние в км после которого вес равен единице
    'one-stop-dist' : 300, # расстояние между соседними о.п. в км
    # подключение к базе UARMS
    'username' : None,
    'password' : None,
}

In [15]:
def main_args_C1(args):
#     args = args_C1
    # открываем лог-файл
    if args['log'] is not None:
        outstream = open('%s/%s' % (args['results'], args['log']), mode='a') 
    else:
        outstream = None
    # выводим информацию о запуске расчета
    print('%s -- запуск расчета с параметрами:' % pd.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), file=outstream)
    for arg in args:
        if (arg != 'username') & (arg != 'password'):
            print('%15s : %s' % (arg, args[arg]), file=outstream)
    #
    # определяем мвн
    mvn = args['source'].split('/')[-1].split('.')[0]
    #
    
    
    #______________
#     # загружаем данные о поостановочных трассах, которые необходимо использовать для получения посадки-высадки
#     print('Загружаем данные о вариантах поостановочных трасс...', file=outstream)
#     try:
#         df_traces = pd.read_excel(args['source'], sheetname='traces')
#         print('...общее число вариантов поостановочных трасс -- %d...' % len(df_traces), file=outstream)
#         try:
#             lst_traces = list(df_traces.query('flag == 1').trace_id.unique())
#             if len(lst_traces) == 0:
#                 print('...варианты поостановочных трассы для построения матрицы не выбраны...', file=outstream)
#                 print('...используем все варианты поостановочных трасс.', file=outstream)    
#             else:
#                 print('...число выбранных вариaнтов поостановочной трассы -- %d.' % len(lst_traces), file=outstream)
#         except:
#             lst_traces = list()
#     except:
#         df_traces = None
#         lst_traces = list()
#         print('...нет информации о вариантах поостановочных трасс.', file=outstream)
    #
    #
    
    
    #______________
    #transpose_stop_id = df_traces['trace'][0]
    #вводим переменную transpose_stop_id, в которую записываем ячейку trace с последовательностью остановок
    #list_transpose = transpose_stop_id.split(',\n')
    #list_transpose
    #list_unstack = list_transpose.unstack()
    #
    
    
    #______________
    # читаем данные о посадке и высадке
    df_asmpp = pd.read_excel(args['source'], sheetname='Sheet1',)# dtype={'pass_in':float,'pass_out':float})
    nasmpp_all = len(df_asmpp)
    # выбираем данные за требуемый период
    #df_asmpp = df_asmpp.query('period_type == "%s"' % (args['period-type']))
    nasmpp_per = len(df_asmpp)
    # выбираем помаршрутные трассы
#     df_asmpp = df_asmpp[df_asmpp.trace_id.isin(lst_traces)]
    if len(df_asmpp) > 0:
        print('Поостановная трасса для построения матрицы:', file=outstream)
        # формируем таблицу с названием о.п. на случай если помаршрутные трассы отличаются
        df_stops = df_asmpp[['stop_sequence','stop_id','stop_name']]\
            .drop_duplicates(subset=['stop_sequence']).sort_values('stop_sequence')
        print(df_stops.set_index('stop_sequence'), file=outstream)
        # определяем число рейсов по каждому из вариантов поостановочной трассы
        print('Загружаем данные о числе рейсов по указанным вариантам поостановочной трассы...', file=outstream)
#         try:
#             # читаем данные о посадке и высадке за требуемый период
#             df_nraces = pd.read_excel(args['source'], sheetname='nraces')\
#                 .query('season == "%s" and workday == "%s" and time == "%s-%s"' % tuple(args['period-type'].split('-')))\
#                 .set_index(['season','workday','time'])\
#                 .stack()\
#                 .reset_index()\
#                 .rename(columns={'level_3':'trace_id',0:'nraces'})
#             # выбираем только указанные варианты поостановочной трассы
#             df_nraces = df_nraces[df_nraces.trace_id.isin(lst_traces)]
#             # добавляем в таблицу с данными асмпп информацию о числе рейсов
#             df_asmpp = df_asmpp.join(df_nraces.set_index('trace_id')['nraces'], on='trace_id')
#             # вычисляем посадку и высадку с учетом числа рейсов
#             df_asmpp['pass_in']  = df_asmpp.eval('pass_in  * nraces')
#             df_asmpp['pass_out'] = df_asmpp.eval('pass_out * nraces')
#             # определяем число рейсов
#             nraces = df_nraces.nraces.sum()
#             print('...число рейсов -- %d.' % nraces, file=outstream)
#         except:
#             print('...нет данных о числе рейсов....', file=outstream)
#             print('...считаем, что по каждому варианту был совершен один рейс.', file=outstream)
#             nraces = len(lst_traces)
        # определяем исходные посадку и высадку
        print('Загружаем исходные данные о посадке-высадке:', file=outstream)
#         df_asmpp = df_stops.join(df_asmpp.groupby('stop_sequence').sum()[['pass_in','pass_out']]/nraces, on='stop_sequence')
        print(df_asmpp.set_index('stop_sequence')[['stop_id','pass_in','pass_out']].round(1), file=outstream)
        print('Суммарно за рейс:', file=outstream)
        print('pass_in  %6.1f' % df_asmpp.pass_in.sum(), file=outstream)
        print('pass_out %6.1f' % df_asmpp.pass_out.sum(), file=outstream)
    else:
        nraces = 0
        df_stops = None
        print('Загружаем исходные данные о посадке-высадке...', file=outstream)
        print('...общее число записей в данных асмпп -- %d...' % nasmpp_all, file=outstream)
        if nasmpp_per > 0:
            print('...число записей в данных асмпп за указанный период -- %d...' % nasmpp_per, file=outstream)
            print('...за указанный период отсутствуют данные по выбранным вариантам поостановочной трассы.', file=outstream)
        else:
            print('...за указанный период отсутствуют данные асмпп.', file=outstream)
    #
    #
    
    
    #______________
    if len(df_asmpp) > 0:
        print('Вносим поправки в исходные данные для корректного вычисления матрицы...', file=outstream)
        # вносим корректировки с данные о посадке и высадке для корректного вычисления матрицы
        df_inout = df_asmpp.copy().set_index('stop_sequence')
        # определяем порядковый номер первого о.п.
        ss_min = df_asmpp.stop_sequence.min()
        print('...наименьший порядковый номер о.п. в поостановочной трассе -- %d...' % ss_min, file=outstream)
        # обнуляем высадку на первом о.п.
        if df_inout.loc[ss_min, 'pass_out'] != 0:
            print('...высадка на начальном о.п. составляет %.1f -- приравниваем нулю...' % df_inout.loc[ss_min, 'pass_out'],
                  file=outstream)
            df_inout.loc[ss_min, 'pass_out'] = 0
        # определяем порядковый номер последнего о.п.
        ss_max = df_asmpp.stop_sequence.max()
        print('...наибольший порядковый номер о.п. в поостановочной трассе -- %d...' % ss_max, file=outstream)
        # обнуляем посадку на последнем о.п.
        if df_inout.loc[ss_max, 'pass_in'] != 0:
            print('...посадка на последнем о.п. составляет %.1f -- приравниваем нулю...' % df_inout.loc[ss_max, 'pass_in'],
                  file=outstream)
            df_inout.loc[ss_max, 'pass_in'] = 0
        # вычисляем пассажиропоток на перегонах
        df_cumsum = df_inout[['pass_in', 'pass_out']].cumsum()
        df_cumsum['pass_inside'] = df_cumsum.eval('pass_in - pass_out')
        df_inout = df_inout.join(df_cumsum[['pass_inside']])
        # корректируем посадку на первом о.п., при необходимости
        diff = (df_inout.pass_inside - df_inout.pass_in).min()
        if diff < 0:
            print('...зафиксировано %.1f пассажиров в салоне тс, увеличиваем посадку на начальном о.п...' % diff, file=outstream)
            df_inout.loc[ss_min, 'pass_in'] = df_inout.loc[ss_min, 'pass_in'] - diff
            df_inout['pass_inside'] = df_inout['pass_inside'] - diff
        # корректируем высадку на последнем о.п., при необходимости
        if df_inout.loc[ss_max, 'pass_inside'] > 0:
            print('...зафиксировано %.1f пассажиров в салоне тс по окончании рейса, увеличиваем высадку на последнем о.п...'
                      % df_inout.loc[ss_max, 'pass_inside'], file=outstream)
            df_inout.loc[ss_max, 'pass_out'] = df_inout.loc[ss_max, 'pass_out'] + df_inout.loc[ss_max, 'pass_inside']
            df_inout.loc[ss_max, 'pass_inside'] = 0
        df_inout.reset_index(inplace=True)
        print('...корректрировка выполнена:', file=outstream)
        print(df_inout.set_index('stop_sequence')[['stop_id','pass_in','pass_out']].round(1), file=outstream)
        print('Суммарно за рейс:', file=outstream)
        print('pass_in  %6.1f' % df_inout.pass_in.sum(), file=outstream)
        print('pass_out %6.1f' % df_inout.pass_out.sum(), file=outstream)
    else:
        df_inout = None
    #
    #
    
    
    #______________
    # подгружаем данные о координатах о.п.
    if df_stops is not None:
        print('Загружаем данные о местоположении о.п...', file=outstream)
        # формируем список уникальных идентификаторов о.п.
        lst_stops = list(df_stops.stop_id.dropna().unique())
        if len(lst_stops) > 0:
            print('...для определения местоположения о.п. подключаемся к базе UARMS...', file=outstream)
            try:
                # инициализируем подключение к базе данных
                cnx = create_engine('postgresql://%s:%s@airms.mgtniip.ru:5432/UARMS'
                    % (args['username'], args['password']))
                # загружаем данные из базы
                gdf_stops = gpd.read_postgis(\
                    "SELECT stop_id,geometry FROM routes.stops_latest as stops WHERE stops.stop_id IN%s" \
                    % (str(tuple(lst_stops)) if len(lst_stops) > 1 else '(' + str(lst_stops[0]) + ')'),\
                    con=cnx, geom_col='geometry')
                if len(gdf_stops) > 0:
                    print('...загружены данные о местоположении %d уникальных о.п.' % len(gdf_stops), file=outstream)
                    # задаем проекцию и проецируем точки на плоскость для вычисляения расстояний по прямой
                    gdf_stops.crs = from_epsg(4326)
                    gdf_stops = gdf_stops.to_crs(from_epsg(32637))
                    # создаем поля с координатами о.п.
                    gdf_stops[['x','y']] = gdf_stops.geometry.bounds[['minx','miny']]
                    gdf_stops.drop('geometry', axis=1, inplace=True)
                    gdf_stops.set_index('stop_id', inplace=True)
                else:
                    print('...в базе нет информации о местоположении ни одного о.п.', file=outstream)
            except:
                print('...не могу загрузить данные из базы -- отсутсвует соединение или неверные логин/пароль.', file=outstream)
                gdf_stops = None
        else:
            print('...в поостановочной трассе отсутствует информация об идентификаторах о.п.', file=outstream)
            gdf_stops = None
    #
    #
    
    
    #______________
    # программная реализация прохода "с двух сторон"
    if df_inout is not None:
        print('Запускаем процедуру вычисления маршрутной матрицы корреспонденций...', file=outstream)  
        print('...инициализация матриц...', file=outstream)  
        # инициализируем матрицы
        start_stop_name = []
        start_stop_id = []
        start_stop_ss = []
        end_stop_name = []
        end_stop_id = []
        end_stop_ss = []
        weight = []
        corr_value = []
        for i, i_id, i_name in zip(df_inout.stop_sequence, df_inout.stop_id, df_inout.stop_name):
            for j, j_id, j_name in zip(df_inout.stop_sequence, df_inout.stop_id, df_inout.stop_name):
                start_stop_ss.append(i)
                start_stop_id.append(i_id)
                end_stop_ss.append(j)
                end_stop_id.append(j_id)
                start_stop_name.append(i_name)
                end_stop_name.append(j_name)
                # инициализируем маршрутную матрицу корреспонденций
                corr_value.append(np.nan)
                # инициализируем матрицу весов
                # определяем расстояние между о.п.
                try:
                    stop2stop_distance =\
                        ( (gdf_stops.loc[i, 'x'] - gdf_stops.loc[j, 'x'])**2\
                        + (gdf_stops.loc[i, 'y'] - gdf_stops.loc[j, 'y'])**2)**0.5  
                except:
                    stop2stop_distance = abs(j-i)*args['one-stop-dist']
                weight.append(min((stop2stop_distance/args['beta'])**args['alpha'], 1))
        df_matrix = pd.DataFrame({'start_num': start_stop_ss,
                                  'start_id' : start_stop_id,
                                  'start_name': start_stop_name,
                                  'end_num': end_stop_ss,
                                  'end_id' : end_stop_id,
                                  'end_name': end_stop_name,
                                  'weight' : weight,
                                  'value': corr_value}).set_index(['start_num','end_num'])
        print('...инициализация векторов...', file=outstream)  
        # инициализация векторов
        inleft = df_inout.set_index('stop_sequence')['pass_in']
        outleft = df_inout.set_index('stop_sequence')['pass_out']
        print('...инициализация переменных...', file=outstream)  
        # инициализация переменных
        lst_ss = list(df_inout.stop_sequence)
        #n = len(lst_ss)
        i = lst_ss[-1]
        # шаг интерации
        print('...расчет матрицы...', file=outstream)  
        for j in lst_ss[1:]:
            # первый этап - распределяем высадку на j-ом о.п.
            # определяем индекс порядкового номера о.п., до которого идет распределение посадки
            m = min(lst_ss.index(j)-1, lst_ss.index(i)-1)
            # опредеяем максимальный вес
            try:
                w_max = df_matrix.query('start_num <= %d and end_num == %d' % (lst_ss[m], j)).weight.max()
            except:
                w_max = 1
            # вычисляем эффективную высадку
            eff = 0
            inleftsum = inleft.loc[:lst_ss[m]].sum()
            for k in lst_ss[:(m+1)]:
                w = df_matrix.loc[(k,j),'weight']
                df_matrix.loc[(k,j),'value'] = (outleft[j] * (w / w_max) * (inleft[k] / inleftsum)) if (inleftsum > 0) else 0
                eff = eff + df_matrix.loc[(k,j),'value']
            # нормируем высадку
            for k in lst_ss[:(m+1)]:
                x = df_matrix.loc[(k,j),'value'] * outleft[j] / eff if eff > 0 else 0
                # проверяем что высадка не превосходит нераспределенную посадку
                if x > inleft[k]:
                    x = inleft[k]
                # обновляем нераспределенную посадку и высадку
                inleft[k] = inleft[k] - x
                outleft[j] = outleft[j] - x
                df_matrix.loc[(k,j),'value'] = x
                # пересчитываем эффективную высадку
                eff = 0
                for l in lst_ss[(lst_ss.index(k)+1):(m+1)]:
                    eff = eff + df_matrix.loc[(l,j),'value']
            # ----------------------
            # второй этап итерации - распределяем посадку i-ого о.п.
            i = lst_ss[-1-lst_ss.index(j)]
            # определяем индекс порядкового номера о.п., с которого идет распределение высадки
            m = max(lst_ss.index(j)+1, lst_ss.index(i)+1)
            # опредеяем максимальный вес
            try:
                w_max = df_matrix.query('start_num == %d and end_num >= %d' % (i, lst_ss[m])).weight.max()
            except:
                w_max = 1
            # вычисляем эффективную высадку
            eff = 0
            outleftsum = outleft.loc[lst_ss[m-1]:].sum()
            for k in lst_ss[:(m-1):-1]:
                w = df_matrix.loc[(i,k),'weight']
                df_matrix.loc[(i,k),'value'] = (inleft[i] * (w / w_max) * (outleft[k] / outleftsum)) if (outleftsum > 0) else 0
                eff = eff + df_matrix.loc[(i,k),'value']
            # нормируем высадку
            for k in lst_ss[:(m-1):-1]:
                x = df_matrix.loc[(i,k),'value'] * inleft[i] / eff if eff > 0 else 0
                # проверяем что высадка не превосходит нераспределенную посадку
                if x > outleft[k]:
                    x = outleft[k]
                # обновляем нераспределенную посадку и высадку
                inleft[i] = inleft[i] - x
                outleft[k] = outleft[k] - x
                df_matrix.loc[(i,k),'value'] = x
                # пересчитываем эффективную высадку
                eff = 0
                for l in lst_ss[(lst_ss.index(k)-1):(m-1):-1]:
                    eff = eff + df_matrix.loc[(i,l),'value']  
            # пишем лог
            print('...о.п. отпр. %3d -- о.п. приб. %3d -- нераспр. посадка %6.2f -- нераспр. высадка %6.2f...' 
                  % (i, j, inleft.sum(), outleft.sum()), file=outstream)
        df_matrix.reset_index(inplace=True)
        df_matrix = df_matrix.drop('weight', axis=1)
        print('...расчет выполнен.', file=outstream)
    else:
        df_matrix = None
    #
    #
    
    
    #______________
    if df_matrix is not None:
        print('Проверяем контрольные суммы по посадке и высадке...', file=outstream)
        # проверяем, что поостановочная посадка и высадка совпадают
        df_compare = df_inout.join(df_matrix.groupby('start_num').sum()[['value']], on='stop_sequence')\
            .join(df_matrix.groupby('end_num').sum()[['value']], on='stop_sequence', lsuffix='_ent', rsuffix='_ext')\
            .fillna(value=0).round(2)

        df_diff_ent = df_compare[df_compare.pass_in  != df_compare.value_ent]
        df_diff_ext = df_compare[df_compare.pass_out != df_compare.value_ext]
        if len(df_diff_ent) > 0:
            print('...данные по посадке отличаются на следующих о.п.:', file=outstream)
            print(df_diff_ent.set_index('stop_sequence')[['pass_in','value_ent']], file=outstream)
        if len(df_diff_ext) > 0:
            print('...данные по высадке отличаются на следующих о.п.:', file=outstream)
            print(df_diff_ext.set_index('stop_sequence')[['pass_out','value_ext']], file=outstream)
        if len(df_diff_ent) + len(df_diff_ext) == 0:
            print('...поостановочные данные по посадке и высадке совпадают.', file=outstream)
    #
    #
    
    
    #______________
    if df_matrix is not None:
        # подготавливаем таблицу для экспорта
        df_table = \
        df_matrix.fillna(value='').rename(columns={'start_name':'Наименование о.п.', 'value':'# о.п. приб.',\
        'start_num':'# о.п. отпр.', 'end_num':mvn, 'start_id':'Идентиф. # о.п.'})\
        .set_index(['# о.п. отпр.', 'Идентиф. # о.п.', 'Наименование о.п.', mvn])[['# о.п. приб.']].unstack()
    #
    #
    
    
    #______________
    # меняем формат значений идентификаторов о.п.
    for fld in ['start_id', 'end_id']:
        df_matrix[fld] = df_matrix[fld].fillna(value=-1).astype(int).replace(-1,'')
    df_inout['stop_id'] = df_inout['stop_id'].fillna(value=-1).astype(int).replace(-1,'')
    df_asmpp['stop_id'] = df_asmpp['stop_id'].fillna(value=-1).astype(int).replace(-1,'')
    #
    
    
    #______________
    if df_matrix is not None:
        print('Сохраняем результаты расчетов в файл %s/%s.xlsx...' % (args['results'], mvn), file=outstream)
        try:
            xls_writer = pd.ExcelWriter(args['results'] + '/'+ mvn + '.xlsx')
            # сохраняем вкладку исх-посадка-высадка
            print('...создаем вкладку "исх-посадка-высадка"...', file=outstream)
            df_asmpp[['stop_id','stop_name','stop_sequence','pass_in','pass_out']]\
                .fillna(value='')\
                .rename(columns={'stop_name':'Наименование о.п.', 'stop_sequence':'# о.п.',\
                                 'pass_in':'Посадка','pass_out':'Высадка','stop_id':'Идентиф. # о.п.'})\
                .set_index(['# о.п.', 'Идентиф. # о.п.', 'Наименование о.п.'])\
                .to_excel(xls_writer, sheet_name='исх-посадка-высадка')
            xls_sheet = xls_writer.sheets['исх-посадка-высадка']
            xls_fmt2 = xls_writer.book.add_format(properties={'num_format': '0.0'})
            xls_sheet.set_column('D:F', 9, xls_fmt2)
            xls_sheet.set_column('C:C', df_matrix.start_name.str.len().max()+4, None)
            xls_sheet.set_column('B:B', 18, None)
            # сохраняем вкладку посадка-высадка
            print('...создаем вкладку "посадка-высадка"...', file=outstream)
            df_inout[['stop_id','stop_name','stop_sequence','pass_in','pass_out']]\
                .fillna(value='')\
                .rename(columns={'stop_name':'Наименование о.п.', 'stop_sequence':'# о.п.',\
                                 'pass_in':'Посадка','pass_out':'Высадка','stop_id':'Идентиф. # о.п.'})\
                .set_index(['# о.п.', 'Идентиф. # о.п.', 'Наименование о.п.'])\
                .to_excel(xls_writer, sheet_name='посадка-высадка')
            xls_sheet = xls_writer.sheets['посадка-высадка']
            xls_fmt2 = xls_writer.book.add_format(properties={'num_format': '0.0'})
            xls_sheet.set_column('D:F', 9, xls_fmt2)
            xls_sheet.set_column('C:C', df_matrix.start_name.str.len().max()+4, None)
            xls_sheet.set_column('B:B', 18, None)
            # сохраняем вкладку матрица
            print('...создаем вкладку "матрица"...', file=outstream)
            df_table.to_excel(xls_writer, sheet_name='матрица')
            xls_sheet = xls_writer.sheets['матрица']
            xls_fmt = xls_writer.book.add_format(properties={'num_format': '0.000'})
            xls_sheet.set_column('A:ZZ', 7, xls_fmt)
            xls_sheet.set_column('A:A', 12, None)
            xls_sheet.set_column('B:B', 18, None)
            xls_sheet.set_column('C:C', df_matrix.start_name.str.len().max()+4, None)
            # сохраняем и закрываем файл
            xls_writer.save()
            print('...сохранение результатов завершено.', file=outstream)
        except:
            print('...при записи файла возникла ошибка!', file=outstream)
    #
    #
    
    
    #______________
    # закрываем лог-файл
    if args['log'] is not None:
        print('----------------------------------------------------------------------------------------------------',
              file=outstream)
        outstream.close()
    #

    return (nraces, df_asmpp, df_matrix)

In [16]:
call_func_C1 = main_args_C1(args)
call_func_C1

2019-02-06 08:55:40 -- запуск расчета с параметрами:
         source : C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/mvns_files/1+Обратное.xlsx
        results : C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/обработанные/преобразованные Вечер пик/
    period-type : None
            log : None
          alpha : 2
           beta : 1000
  one-stop-dist : 300
Поостановная трасса для построения матрицы:
               stop_id                      stop_name
stop_sequence                                        
1                  785          Городское кладбище №3
2                  757                  Осиновая гора
3                  738          Городское кладбище №2
4                  708          Городское кладбище №1
5                  710              Автоколонна №1136
6                  700              Автоколонна №1809
7                  714              Спецавтохозяйство
8          

C:\Users\Popova_KV\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:1310: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\Users\Popova_KV\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: PerformanceWarning: indexing past lexsort depth may impact performance.
  interactivity=interactivity, compiler=compiler, result=result)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Part C2

In [10]:
# задаем директорию с исходными данными
args['source'] = 'C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/mvns_files/'
path = args['source']


# !!!!!!
# задаем директорию с результатами
args['results'] = ('C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/обработанные/преобразованные '
+ timing +'/')


# логин и пароль для подключения к базе UARMS
args['username'] = None
args['password'] = None
args['period-type'] = None
args

{'alpha': 2,
 'beta': 1000,
 'log': None,
 'one-stop-dist': 300,
 'password': None,
 'period-type': None,
 'results': 'C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/обработанные/преобразованные Вечер пик/',
 'source': 'C:/Users/Popova_KV/Desktop/Новая папка/Scripts/matrix_from_detour/Tula/for_tests/for_avg_matrix/mvns_files/',
 'username': None}

In [11]:
# запускаем расчет для всех файлов в директории включая вложенные директории

# запустил для всех мвн 
# (из апи мосгортранса обработаны архивы с 20161220 по 20180228)
# (по коммерческим перевозчикам обработаны данные за 4 кв 2017 года)
# (дополнительно подготовлдены данные по некоторым маршрутам мосгортранса, которых нет в апи)
# время выполнения составляет ЧЧ:ММ:СС (1977 мвн'ов)
# - 1 мвн обработано с ошибкой (0.1%)
# - 486 мвн'у не было совершено ни одного рейса (24.5%)
# - и только 1054 мвн'ов имеют 5 и более рейсов (53.3%)

ts_start = pd.datetime.now()
lst_mvns = list()
lst_nraces = list()
df_matrix = None
df_inout = None
for file in tqdm(os.listdir(path=path)):
    
    
    # !!!!!!
    
    
    if (file.split('.')[-1] == 'xlsx') & (file.split('.')[0].split('-')[-1] == timing):
        mvn = file[:-5]
        lst_mvns.append(mvn)
        args['source'] = path + '/' + file
        args['log'] = mvn + '.log'
        try:
            # (число рейсов, исх-посадка-высадка, матрица)
            (n, inout, matrix) = main_args_C1(args)
            # добавляем данные об исходной посадке-высадке
            if inout is not None:
                inout['mvn'] = mvn
            if df_inout is None:
                    df_inout = inout
            else:
                df_inout = pd.concat([df_inout, inout], ignore_index=True)
            # добавляем данные по поездкам между о.п.
            if matrix is not None:
                matrix['mvn'] = mvn
            if df_matrix is None:
                df_matrix = matrix
            else:
                df_matrix = pd.concat([df_matrix, matrix], ignore_index=True)
            lst_nraces.append(n)
        except:
            lst_nraces.append('ERROR')

In [ ]:
df_results = pd.DataFrame({'mvn': lst_mvns, 'nraces': lst_nraces})
df_results.to_csv(args['results'] + '/log' + ts_start.strftime('-%Y-%m-%d-%H-%M')+ '.csv', sep=';', index=False)

In [ ]:
df_matrix\
    .rename(columns={'start_num':'departure_stop_seq', 'start_id':'departure_stop_id',\
        'end_num':'arrival_stop_seq', 'end_id':'arrival_stop_id', 'value' : 'pass'})\
    [['mvn','departure_stop_seq', 'departure_stop_id', 'arrival_stop_seq', 'arrival_stop_id', 'pass']]\
    .dropna(subset=['pass'])\
    .sort_values(['mvn','departure_stop_seq','arrival_stop_seq'])\
    .to_csv('%s/matrix.csv' % args['results'], sep=';', encoding='cp1251', float_format='%.3f', index=False)

In [ ]:
df_inout\
    [['stop_id','mvn','stop_sequence','pass_in','pass_out']]\
    .sort_values(['stop_id','mvn','stop_sequence'])\
    .to_csv('%s/inout.csv' % args['results'], sep=';', encoding='cp1251', float_format='%.3f', index=False)

In [ ]:
df_matrix.groupby(['start_id','end_id'])[['value']].sum().to_csv('%s/matrix_summed.csv' % args['results'], sep=';', float_format='%.3f')